In [1]:
import sys
import pandas as pd
from Bio import SeqIO

test_reads_r = "/mnt/c/seqdata_illumina_amplicon/runs_2023/10_20230804_bonobo-3/results-mgs-pipeline/kraken/reads/BVF-065-4_S330_classified"
test_reads_1 = f'{test_reads_r}__1.fq'
test_reads_2 = f'{test_reads_r}__2.fq'
test_report = "/mnt/c/seqdata_illumina_amplicon/runs_2023/10_20230804_bonobo-3/results-mgs-pipeline/kraken/BVF-065-4_S330.kraken2.report"

taxonomy_file = "/media/ramdisk/krakendb/taxonomy/nodes.dmp"

test_report_bracken = "/mnt/c/seqdata_illumina_amplicon/runs_2023/10_20230804_bonobo-3/results-mgs-pipeline/bracken/BVF-065-4_S330.bracken.report"
test_output_bracken = "/mnt/c/seqdata_illumina_amplicon/runs_2023/10_20230804_bonobo-3/results-mgs-pipeline/bracken/raw/BVF-065-4_S330.bracken.out"
classif = pd.read_table(test_report_bracken, names=["total_perc", "reads_tot", "reads_spec", "level", "taxID", "taxName"])
classif["sequence"] = ""

def extract_asvs_from_reads_file(reads_file, classif, taxid_field="taxID"):
    
    records = SeqIO.to_dict(SeqIO.parse(reads_file, "fastq"))

    for _, item in records.items():
        taxid = int(item.description.split("kraken:taxid|")[1])
        sequence = item.seq
        
        if len(sequence) > len(classif.loc[classif[taxid_field] == taxid, ["sequence"]]):
            classif.loc[classif[taxid_field] == taxid, ["sequence"]] = str(sequence)
    
    return classif


for f in [test_reads_1, test_reads_2]:
    classif = extract_asvs_from_reads_file(f, classif)


In [10]:
seqs = pd.read_table("/mnt/c/seqdata_illumina_amplicon/runs_2023/10_20230804_bonobo-3/results-mgs-pipeline/bracken/sequences/allseqs", names=["taxid","seq"])

seqs.groupby("taxid").first()

,seq
taxid,
2,AACGTAGGGTGCAAGCGTTGTCCGGAATTACTGGGTGTAAAGGGAG...
3,TACGTAGGGGGCAAGCGTTGTCCGGATTGATTGGGCGTAAAGGGCG...
4,TACGTAGGGTGCAAGCGTTAATCGGAATTACTGGGCGTAAAGCGTG...
48836,TACGTAGGTGGCGAGCGTTATCCGGAATCATTGGGCGTAAAGGGTG...
77634,TACGTAGGTGGCAAGCGTTATCCGGATTTATTGGGCGTAAAGGGAA...
...,...
271649,TACGGAGACTACGAGCGTTACTCGGATTCACTGGGCGTAAAGGGAG...
271650,TACGGAGACTACGAGCGTTACTCGGATTCACTGGGCGTAAAGGGAG...
278088,TACGTAGGTGGCAAGCGTTGTCCGGAATGACTGGGCGTAAAGGGAG...


In [66]:
sequences_kraken = classif[classif.sequence != ""]
classif_bracken = pd.read_table(test_report_bracken, names=["total_perc", "reads_tot", "reads_spec", "level", "taxID", "taxName"])

## Reads assigned to S1 can't extract ASVs as the taxid used in classified_out is that of the S1
## => use taxonomy files to retrace origin S1's and link them to S taxID 

sequences_kraken
class_bracken = classif_bracken.merge(sequences_kraken[["taxID","sequence"]], on="taxID", how="left")

class_bracken[pd.isna(class_bracken["sequence"])]

,total_perc,reads_tot,reads_spec,level,taxID,taxName,sequence
2,39.95,15688,0,P,107627,p__Bacillota_A,NaN
7,2.20,865,865,S,254017,s__Blautia_A sp900066145,NaN
8,1.18,462,462,S,217892,s__Blautia_A faecis,NaN
9,4.35,1706,1706,S,286135,s__Blautia_A luti,NaN
11,1.95,767,767,S,274833,s__Lachnoclostridium_B sp900066555,NaN
...,...,...,...,...,...,...,...
156,0.09,37,37,S,268775,s__Treponema_D berlinense,NaN
157,0.09,34,0,P,208014,p__Cyanobacteriota,NaN
158,0.09,34,0,C,238101,c__Vampirovibrionia,NaN
161,0.09,34,0,G,300915,g__UBA2883,NaN
